In [1]:
### ライブラリのインポート
import datatable
import pandas as pd
import numpy as np

import datetime as dt
import jpholiday

pd.set_option('max_columns', 5000)
pd.set_option('max_rows', 5000)

import gc
from IPython.display import display

import warnings
warnings.filterwarnings('ignore')

In [2]:
### データの読込
train = datatable.fread("data/sales_train_evaluation.csv")
train = train.to_pandas()
train = train.iloc[:, :35]
print("sales_train_evaluation")
display(train.head(1))

cal = pd.read_csv("data/calendar.csv")
cal = cal[["d", "date", "weekday"]]
print("calendar")
display(cal.head(5))

sales_train_evaluation


,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


calendar


,d,date,weekday
0,d_1,2011-01-29,Saturday
1,d_2,2011-01-30,Sunday
2,d_3,2011-01-31,Monday
3,d_4,2011-02-01,Tuesday
4,d_5,2011-02-02,Wednesday


In [3]:
# 各列ごとの日付データを1列に取り込む
# ラグ作成のためには1日1個しかない カラム が必要
# なければ作成する たとえば アイテムコード + 店舗コード で作成可能
dtable = pd.melt(train, id_vars=train.columns.values[:6],
    var_name="d", value_name="sales_quantity")

dtable = dtable[["id", "cat_id", "item_id", "store_id", "d", "sales_quantity"]]

In [4]:
# メモリ開放
del train
gc.collect()

40

In [5]:
# データのマージ
dtable = pd.merge(cal, dtable, on="d")

dtable = dtable.iloc[:, 1:]

dtable.head(1)

,date,weekday,id,cat_id,item_id,store_id,sales_quantity
0,2011-01-29,Saturday,HOBBIES_1_001_CA_1_evaluation,HOBBIES,HOBBIES_1_001,CA_1,0


In [6]:
# メモリ開放
del cal
gc.collect()

40

In [7]:
# date 列を日付データに変換
dtable["date"] = pd.to_datetime(dtable["date"])

# 仮に予測データを5で仮置きする
dtable["pred"] = 5

In [8]:
# 年
dtable["year"] = dtable["date"].dt.year

# 月
dtable["month"] = dtable["date"].dt.month

# 週番号
dtable["numofweek"] = dtable["date"].dt.strftime("%V")

# 日
dtable["day"] = dtable["date"].dt.day

# 曜日 月曜が0, 日曜が6
dtable["dayofweek"] = dtable["date"].dt.dayofweek

In [91]:
# 祝日の判定 重い
dtable["p_holiday"] = dtable["date"].map(jpholiday.is_holiday).astype(int)

In [92]:
# 土日、祝日を1、それ以外を0とする
dtable["holiday"]=0
dtable.loc[(dtable["dayofweek"]==5)|(dtable["dayofweek"]==6)|(dtable["p_holiday"]==1), 'holiday'] = 1

In [147]:
dtable.head()

,date,weekday,id,cat_id,item_id,store_id,sales_quantity,pred,year,month,day,dayofweek,p_holiday,holiday,lag_1,lag_7,lag_1_diff,lag_7_diff,rmean_1_1,rmean_7_1,rmean_1_7,rmean_7_7,store_item_id,numofweek
0,2011-01-29,Saturday,HOBBIES_1_001_CA_1_evaluation,HOBBIES,HOBBIES_1_001,CA_1,0,5,2011,1,29,5,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CA_1-HOBBIES_1_001,04
1,2011-01-29,Saturday,HOBBIES_1_002_CA_1_evaluation,HOBBIES,HOBBIES_1_002,CA_1,0,5,2011,1,29,5,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CA_1-HOBBIES_1_002,04
2,2011-01-29,Saturday,HOBBIES_1_003_CA_1_evaluation,HOBBIES,HOBBIES_1_003,CA_1,0,5,2011,1,29,5,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CA_1-HOBBIES_1_003,04
3,2011-01-29,Saturday,HOBBIES_1_004_CA_1_evaluation,HOBBIES,HOBBIES_1_004,CA_1,0,5,2011,1,29,5,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CA_1-HOBBIES_1_004,04
4,2011-01-29,Saturday,HOBBIES_1_005_CA_1_evaluation,HOBBIES,HOBBIES_1_005,CA_1,0,5,2011,1,29,5,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CA_1-HOBBIES_1_005,04


### 週単位に集約する

In [ ]:
week_pivot = pd.pivot_table(dtable, values="sales_quantity", 
    index=["numofweek", "weekday", "id", "cat_id", "item_id", "store_id", "year", "month", "store_item_id" ], aggfunc='sum')

week_pivot.head()

In [ ]:
# マルチインデックスになっているので、解除する
week_pivot = week_pivot.reset_index()

week_pivot.head()

### 時系列解析用のラグデータを作成

In [95]:
# 1日1個しかない id ごとに ラグデータを作成する
# id 以外は1日複数個あるのでNG
# item_id は店舗の数だけあるのでNG
dayLags = [1, 7]
lagSalesCols = [f"lag_{dayLag}" for dayLag in dayLags]

for dayLag, lagSalesCol in zip(dayLags, lagSalesCols):
    dtable[lagSalesCol] = dtable[["id","sales_quantity"]].groupby("id")["sales_quantity"].shift(dayLag)

In [96]:
# ラグの差分を作る
for dayLag, lagSalesCol in zip(dayLags, lagSalesCols):
    dtable[f"{lagSalesCol}_diff"] = dtable[["id",f"lag_{dayLag}"]].groupby("id")[f"lag_{dayLag}"].diff(dayLag)

In [97]:
# 平均値も作る
windows = [1, 7]
for window in windows:
    for dayLag, lagSalesCol in zip(dayLags, lagSalesCols):
        dtable[f"rmean_{dayLag}_{window}"] = dtable[["id", lagSalesCol]].groupby("id")[lagSalesCol].transform(lambda x: x.rolling(window).mean())

In [99]:
# 店舗ごとの集計
dtable.groupby("store_id").sum()

,sales_quantity,pred,year,month,day,dayofweek,p_holiday,holiday,lag_1,lag_7,lag_1_diff,lag_7_diff,rmean_1_1,rmean_7_1,rmean_1_7,rmean_7_7
store_id,,,,,,,,,,,,,,,,
CA_1,94948,442105,177814631,167695,1344609,271361,3049,30490,91229.0,74066.0,-1636.0,-2325.0,91229.0,74066.0,72464.571429,53978.571429
CA_2,71761,442105,177814631,167695,1344609,271361,3049,30490,68245.0,55057.0,-1307.0,-2034.0,68245.0,55057.0,53912.571429,40136.142857
CA_3,124999,442105,177814631,167695,1344609,271361,3049,30490,120225.0,96980.0,-1164.0,-2877.0,120225.0,96980.0,96137.714286,72507.142857
CA_4,42814,442105,177814631,167695,1344609,271361,3049,30490,41171.0,32873.0,-241.0,-673.0,41171.0,32873.0,32468.714286,24040.857143
TX_1,64926,442105,177814631,167695,1344609,271361,3049,30490,62297.0,50049.0,-670.0,433.0,62297.0,50049.0,49577.142857,36668.857143
TX_2,89212,442105,177814631,167695,1344609,271361,3049,30490,85647.0,68809.0,-1346.0,-233.0,85647.0,68809.0,67706.142857,49886.428571
TX_3,68082,442105,177814631,167695,1344609,271361,3049,30490,65690.0,52787.0,-1136.0,212.0,65690.0,52787.0,51925.428571,37886.428571
WI_1,49941,442105,177814631,167695,1344609,271361,3049,30490,47787.0,39652.0,-874.0,-453.0,47787.0,39652.0,37832.000000,28485.142857
WI_2,54724,442105,177814631,167695,1344609,271361,3049,30490,52763.0,43663.0,-724.0,-1308.0,52763.0,43663.0,41963.714286,31992.285714


In [123]:
# item_id の指定
limited_item_id = ['HOBBIES_1_001', 'HOBBIES_1_002', 'HOBBIES_1_003',
'HOBBIES_1_005',  'HOUSEHOLD_1_344',
 'HOUSEHOLD_1_345', 'HOUSEHOLD_1_346',
 ]

dtable[dtable["item_id"].isin(limited_item_id)].head()

,date,weekday,id,cat_id,item_id,store_id,sales_quantity,pred,year,month,day,dayofweek,p_holiday,holiday,lag_1,lag_7,lag_1_diff,lag_7_diff,rmean_1_1,rmean_7_1,rmean_1_7,rmean_7_7,store_item_id,numofweek
0,2011-01-29,Saturday,HOBBIES_1_001_CA_1_evaluation,HOBBIES,HOBBIES_1_001,CA_1,0,5,2011,1,29,5,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CA_1-HOBBIES_1_001,04
1,2011-01-29,Saturday,HOBBIES_1_002_CA_1_evaluation,HOBBIES,HOBBIES_1_002,CA_1,0,5,2011,1,29,5,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CA_1-HOBBIES_1_002,04
2,2011-01-29,Saturday,HOBBIES_1_003_CA_1_evaluation,HOBBIES,HOBBIES_1_003,CA_1,0,5,2011,1,29,5,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CA_1-HOBBIES_1_003,04
4,2011-01-29,Saturday,HOBBIES_1_005_CA_1_evaluation,HOBBIES,HOBBIES_1_005,CA_1,0,5,2011,1,29,5,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CA_1-HOBBIES_1_005,04
902,2011-01-29,Saturday,HOUSEHOLD_1_344_CA_1_evaluation,HOUSEHOLD,HOUSEHOLD_1_344,CA_1,0,5,2011,1,29,5,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CA_1-HOUSEHOLD_1_344,04


### 期間で集計

In [124]:

from_years, from_months, from_days = 2011, 2, 15
till_years, till_months, till_days = 2011, 2, 20

# 特定期間
specific_period = dtable[(dtable["date"] >= dt.datetime(from_years, from_months, from_days)) \
& (dtable["date"] <= dt.datetime(till_years, till_months, till_days))]

display(specific_period.head(2))
display(specific_period.tail(2))

,date,weekday,id,cat_id,item_id,store_id,sales_quantity,pred,year,month,day,dayofweek,p_holiday,holiday,lag_1,lag_7,lag_1_diff,lag_7_diff,rmean_1_1,rmean_7_1,rmean_1_7,rmean_7_7,store_item_id,numofweek
518330,2011-02-15,Tuesday,HOBBIES_1_001_CA_1_evaluation,HOBBIES,HOBBIES_1_001,CA_1,0,5,2011,2,15,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CA_1-HOBBIES_1_001,07
518331,2011-02-15,Tuesday,HOBBIES_1_002_CA_1_evaluation,HOBBIES,HOBBIES_1_002,CA_1,0,5,2011,2,15,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CA_1-HOBBIES_1_002,07


,date,weekday,id,cat_id,item_id,store_id,sales_quantity,pred,year,month,day,dayofweek,p_holiday,holiday,lag_1,lag_7,lag_1_diff,lag_7_diff,rmean_1_1,rmean_7_1,rmean_1_7,rmean_7_7,store_item_id,numofweek
701268,2011-02-20,Sunday,FOODS_3_826_WI_3_evaluation,FOODS,FOODS_3_826,WI_3,0,5,2011,2,20,6,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,WI_3-FOODS_3_826,07
701269,2011-02-20,Sunday,FOODS_3_827_WI_3_evaluation,FOODS,FOODS_3_827,WI_3,0,5,2011,2,20,6,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,WI_3-FOODS_3_827,07


### 特定期間の売上TOP 30%以内のitem_idを抽出

In [125]:
item_id_group = pd.DataFrame(specific_period.groupby("item_id").sum()).sort_values('sales_quantity', ascending=False)

# 降順で累積
item_id_group["cum_ratio"] = item_id_group["sales_quantity"].cumsum()/item_id_group["sales_quantity"].sum()

item_id_group.head()

,sales_quantity,pred,year,month,day,dayofweek,p_holiday,holiday,lag_1,lag_7,lag_1_diff,lag_7_diff,rmean_1_1,rmean_7_1,rmean_1_7,rmean_7_7,cum_ratio
item_id,,,,,,,,,,,,,,,,,
FOODS_3_586,2907,300,120660,120,1050,210,0,20,2795.0,3101.0,-105.0,410.0,2795.0,3101.0,3082.714286,2805.714286,0.019313
FOODS_3_587,2023,300,120660,120,1050,210,0,20,1884.0,2345.0,-98.0,346.0,1884.0,2345.0,2173.285714,2335.428571,0.032752
FOODS_3_555,1789,300,120660,120,1050,210,0,20,1691.0,1829.0,-7.0,181.0,1691.0,1829.0,1781.714286,1696.428571,0.044637
FOODS_3_252,1611,300,120660,120,1050,210,0,20,1549.0,1712.0,-62.0,345.0,1549.0,1712.0,1737.571429,1449.714286,0.055340
FOODS_3_694,1416,300,120660,120,1050,210,0,20,1406.0,1439.0,-51.0,39.0,1406.0,1439.0,1482.857143,1349.142857,0.064747


In [126]:
# 売上TOP30%のitem_idのリスト
top30_item_id_list = list(item_id_group[item_id_group["cum_ratio"]<=0.3].index)

In [127]:
# 売上TOP30%のitem_idを含む項目のみ表示
dtable[dtable["item_id"].isin(top30_item_id_list)].head()

,date,weekday,id,cat_id,item_id,store_id,sales_quantity,pred,year,month,day,dayofweek,p_holiday,holiday,lag_1,lag_7,lag_1_diff,lag_7_diff,rmean_1_1,rmean_7_1,rmean_1_7,rmean_7_7,store_item_id,numofweek
31,2011-01-29,Saturday,HOBBIES_1_032_CA_1_evaluation,HOBBIES,HOBBIES_1_032,CA_1,9,5,2011,1,29,5,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CA_1-HOBBIES_1_032,04
65,2011-01-29,Saturday,HOBBIES_1_067_CA_1_evaluation,HOBBIES,HOBBIES_1_067,CA_1,0,5,2011,1,29,5,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CA_1-HOBBIES_1_067,04
98,2011-01-29,Saturday,HOBBIES_1_103_CA_1_evaluation,HOBBIES,HOBBIES_1_103,CA_1,25,5,2011,1,29,5,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CA_1-HOBBIES_1_103,04
332,2011-01-29,Saturday,HOBBIES_1_341_CA_1_evaluation,HOBBIES,HOBBIES_1_341,CA_1,50,5,2011,1,29,5,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CA_1-HOBBIES_1_341,04
362,2011-01-29,Saturday,HOBBIES_1_371_CA_1_evaluation,HOBBIES,HOBBIES_1_371,CA_1,14,5,2011,1,29,5,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CA_1-HOBBIES_1_371,04


In [128]:
# トップ30%の合計をcat_idごとに集計
print("トップ30%")
display(dtable[dtable["item_id"].isin(top30_item_id_list)].groupby("cat_id").sum())

# トップ30%以外の合計をcat_idごとに集計
print("トップ30%以外")
display(dtable[~dtable["item_id"].isin(top30_item_id_list)].groupby("cat_id").sum())

# 全部の合計をcat_idごとに集計
print("全部")
display(dtable.groupby("cat_id").sum())

トップ30%


,sales_quantity,pred,year,month,day,dayofweek,p_holiday,holiday,lag_1,lag_7,lag_1_diff,lag_7_diff,rmean_1_1,rmean_7_1,rmean_1_7,rmean_7_7
cat_id,,,,,,,,,,,,,,,,
FOODS,196765,79750,32075450,30250,242550,48950,550,5500,189378.0,153422.0,-1148.0,5253.0,189378.0,153422.0,153191.000000,114151.571429
HOBBIES,10592,7250,2915950,2750,22050,4450,50,500,9979.0,7861.0,-207.0,-449.0,9979.0,7861.0,7753.857143,5647.285714
HOUSEHOLD,9348,7250,2915950,2750,22050,4450,50,500,8827.0,6435.0,10.0,286.0,8827.0,6435.0,6785.571429,4488.571429


トップ30%以外


,sales_quantity,pred,year,month,day,dayofweek,p_holiday,holiday,lag_1,lag_7,lag_1_diff,lag_7_diff,rmean_1_1,rmean_7_1,rmean_1_7,rmean_7_7
cat_id,,,,,,,,,,,,,,,,
FOODS,350869,2003900,805968580,760100,6094620,1229980,13820,138200,338351.0,280718.0,-6550.0,-17158.0,338351.0,280718.0,268586.142857,206822.285714
HOBBIES,72684,812000,326586400,308000,2469600,498400,5600,56000,69209.0,55163.0,-882.0,-1152.0,69209.0,55163.0,54197.857143,39766.571429
HOUSEHOLD,124333,1510900,607683980,573100,4595220,927380,10420,104200,118939.0,93771.0,-1325.0,2703.0,118939.0,93771.0,93924.714286,66824.285714


全部


,sales_quantity,pred,year,month,day,dayofweek,p_holiday,holiday,lag_1,lag_7,lag_1_diff,lag_7_diff,rmean_1_1,rmean_7_1,rmean_1_7,rmean_7_7
cat_id,,,,,,,,,,,,,,,,
FOODS,547634,2083650,838044030,790350,6337170,1278930,14370,143700,527729.0,434140.0,-7698.0,-11905.0,527729.0,434140.0,421777.142857,320973.857143
HOBBIES,83276,819250,329502350,310750,2491650,502850,5650,56500,79188.0,63024.0,-1089.0,-1601.0,79188.0,63024.0,61951.714286,45413.857143
HOUSEHOLD,133681,1518150,610599930,575850,4617270,931830,10470,104700,127766.0,100206.0,-1315.0,2989.0,127766.0,100206.0,100710.285714,71312.857143


### 店舗別にトップ30% item_id を抽出

In [129]:
# store_id、item_idで集計
store_item_id_group_sum = dtable[["store_id", "item_id", "sales_quantity"]].groupby(["store_id", "item_id"]).sum()

store_item_id_group_sum.head()

sales_quantity
store_id item_id                    
CA_1     FOODS_1_001              41
         FOODS_1_002              10
         FOODS_1_003              26
         FOODS_1_004               0
         FOODS_1_005              84

In [130]:
# store_idはそのままに、店舗ごとにitem_idがsales_quantityの降順に並ぶようにソート
store_item_id_group_sum_sort = store_item_id_group_sum.sort_values(["store_id", "sales_quantity"], ascending=[True, False])

store_item_id_group_sum_sort.head()

sales_quantity
store_id item_id                    
CA_1     FOODS_3_587            2211
         FOODS_3_318            1737
         FOODS_3_586            1109
         FOODS_3_714            1095
         FOODS_3_030            1059

In [131]:
# sales_quantityの累計を算定
store_item_id_group_sum_sort_cumsum = store_item_id_group_sum_sort.groupby("store_id").transform(np.cumsum)

store_item_id_group_sum_sort_cumsum.head()

sales_quantity
store_id item_id                    
CA_1     FOODS_3_587            2211
         FOODS_3_318            3948
         FOODS_3_586            5057
         FOODS_3_714            6152
         FOODS_3_030            7211

In [132]:
# 分母と並びを合わせるため、店舗別にitem_idを昇順に並び替える
numerator = store_item_id_group_sum_sort_cumsum.sort_values(["store_id", "item_id"], ascending=[True, True])

numerator.head()

sales_quantity
store_id item_id                    
CA_1     FOODS_1_001           80621
         FOODS_1_002           93201
         FOODS_1_003           87608
         FOODS_1_004           94948
         FOODS_1_005           63935

In [133]:
# store_id、item_idで集計
store_item_id_group_sum = dtable[["store_id", "item_id", "sales_quantity"]].groupby(["store_id", "item_id"]).sum()

store_item_id_group_sum.head()

sales_quantity
store_id item_id                    
CA_1     FOODS_1_001              41
         FOODS_1_002              10
         FOODS_1_003              26
         FOODS_1_004               0
         FOODS_1_005              84

In [134]:
# さらにstore_idごとの合計を算定する → その合計額が各店舗のitem_idに計上される
denominator = store_item_id_group_sum.groupby("store_id").transform(np.sum)

denominator.head()

sales_quantity
store_id item_id                    
CA_1     FOODS_1_001           94948
         FOODS_1_002           94948
         FOODS_1_003           94948
         FOODS_1_004           94948
         FOODS_1_005           94948

In [135]:
# (店舗別のitem_id(降順)の累計) ÷ (各item_idに店舗別の合計を計上したもの)で
# 店舗別のitem_id(降順)の累計比率を算定

by_store_item_id_rate = numerator / denominator

by_store_item_id_rate.head()

sales_quantity
store_id item_id                    
CA_1     FOODS_1_001        0.849107
         FOODS_1_002        0.981600
         FOODS_1_003        0.922695
         FOODS_1_004        1.000000
         FOODS_1_005        0.673369

In [136]:
# 店舗別のitem_id(降順)の累計比率が30%以下の項目を抽出すれば、各店舗ごとのTOP30％のitem_idを抽出可能
by_store_item_id_rate<=0.3

sales_quantity
store_id item_id                        
CA_1     FOODS_1_001               False
         FOODS_1_002               False
         FOODS_1_003               False
         FOODS_1_004               False
         FOODS_1_005               False
...                                  ...
WI_3     HOUSEHOLD_2_512           False
         HOUSEHOLD_2_513           False
         HOUSEHOLD_2_514           False
         HOUSEHOLD_2_515           False
         HOUSEHOLD_2_516           False

[30490 rows x 1 columns]

In [137]:
# 使いやすいよう、マルチインデックスを解除する
by_store_item_id_rate = by_store_item_id_rate.reset_index()

by_store_item_id_rate.head()

,store_id,item_id,sales_quantity
0,CA_1,FOODS_1_001,0.849107
1,CA_1,FOODS_1_002,0.981600
2,CA_1,FOODS_1_003,0.922695
3,CA_1,FOODS_1_004,1.000000
4,CA_1,FOODS_1_005,0.673369


In [138]:
# store_idとitem_idの2つのカラムに分かれていると、DataFrame型の入れ子で使えない
# そこでstore_idとitem_idを組み合わせた store_item_id を作成する
by_store_item_id_rate["store_item_id"] = by_store_item_id_rate["store_id"].str.cat(by_store_item_id_rate["item_id"], sep="-")

# store_item_id を index とする
by_store_item_id_rate = by_store_item_id_rate[["store_item_id", "sales_quantity"]].set_index("store_item_id")

display(by_store_item_id_rate.head())

display(by_store_item_id_rate["sales_quantity"]<0.3)

,sales_quantity
store_item_id,
CA_1-FOODS_1_001,0.849107
CA_1-FOODS_1_002,0.981600
CA_1-FOODS_1_003,0.922695
CA_1-FOODS_1_004,1.000000
CA_1-FOODS_1_005,0.673369


store_item_id
CA_1-FOODS_1_001        False
CA_1-FOODS_1_002        False
CA_1-FOODS_1_003        False
CA_1-FOODS_1_004        False
CA_1-FOODS_1_005        False
                        ...  
WI_3-HOUSEHOLD_2_512    False
WI_3-HOUSEHOLD_2_513    False
WI_3-HOUSEHOLD_2_514    False
WI_3-HOUSEHOLD_2_515    False
WI_3-HOUSEHOLD_2_516    False
Name: sales_quantity, Length: 30490, dtype: bool

In [139]:
# 各店舗ごとのTOP30%のitem_idを表示
byshop_top_30 = by_store_item_id_rate[by_store_item_id_rate["sales_quantity"]<0.3]

display(byshop_top_30.head())

# 各店舗ごとのTOP30%のitem_idのリストを作成
byshop_top_30_list = list(byshop_top_30.index)

,sales_quantity
store_item_id,
CA_1-FOODS_1_032,0.148829
CA_1-FOODS_1_085,0.277878
CA_1-FOODS_2_019,0.236266
CA_1-FOODS_2_021,0.245029
CA_1-FOODS_2_197,0.212601


In [140]:
# 店舗ごとのTOP30％を表示
dtable["store_item_id"] = dtable["store_id"].str.cat(dtable["item_id"], sep="-")

dtable[dtable["store_item_id"].isin(byshop_top_30_list)].head()

,date,weekday,id,cat_id,item_id,store_id,sales_quantity,pred,year,month,day,dayofweek,p_holiday,holiday,lag_1,lag_7,lag_1_diff,lag_7_diff,rmean_1_1,rmean_7_1,rmean_1_7,rmean_7_7,store_item_id,numofweek
98,2011-01-29,Saturday,HOBBIES_1_103_CA_1_evaluation,HOBBIES,HOBBIES_1_103,CA_1,25,5,2011,1,29,5,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CA_1-HOBBIES_1_103,04
164,2011-01-29,Saturday,HOBBIES_1_169_CA_1_evaluation,HOBBIES,HOBBIES_1_169,CA_1,45,5,2011,1,29,5,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CA_1-HOBBIES_1_169,04
248,2011-01-29,Saturday,HOBBIES_1_256_CA_1_evaluation,HOBBIES,HOBBIES_1_256,CA_1,54,5,2011,1,29,5,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CA_1-HOBBIES_1_256,04
260,2011-01-29,Saturday,HOBBIES_1_268_CA_1_evaluation,HOBBIES,HOBBIES_1_268,CA_1,16,5,2011,1,29,5,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CA_1-HOBBIES_1_268,04
270,2011-01-29,Saturday,HOBBIES_1_278_CA_1_evaluation,HOBBIES,HOBBIES_1_278,CA_1,0,5,2011,1,29,5,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CA_1-HOBBIES_1_278,04
